In [ ]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
from datetime import datetime as dt
import re
import warnings
warnings.filterwarnings('ignore')
app = dash.Dash()


###extracting and formatting data
global mob_df
mob_df = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv")
mob_df["sub_region_1"]=mob_df["sub_region_1"].fillna(mob_df['country_region'])
mob_df["sub_region_2"]=mob_df["sub_region_2"].fillna(mob_df['sub_region_1'])
countrylista=mob_df['country_region']
countrylist = list(set(countrylista))

region1lista=mob_df["sub_region_1"].dropna()
region1listb=list(set(region1lista))
region1list=region1listb[1:len(region1listb)]

region2lista=mob_df["sub_region_2"].dropna()
region2listb=list(set(region2lista))
region2list=region2listb[1:len(region2listb)]

countryoption =[]
for i in countrylist:
    countryoption.append({"label":i,"value":i})

region1option1=[]
for i in region1list:
    region1option1.append({"label":i,"value":i})
region1option = countryoption + region1option1

region2option1=[]
for i in region2list:
    region2option1.append({"label":i,"value":i})
region2option = region1option1 + region2option1


app.layout = html.Div(children=[html.Div("Google Mobility Dashboard",style=
                                        {"display":"inline-block","border-style":"dashed",
                                        "text-align":"center","width":"80%","color":"white",
                                        "background-color":"blue","border-color":"white"}),
                                html.Div(dcc.Dropdown(id="dropdown1",
                                                     options=countryoption,value='United States'),style=
                                         {"display":"inline-block","border-style":"dashed",
                                        "text-align":"center","width":"26%","color":"black",
                                        "background-color":"orange","border-color":"white"                                             
                                         }),
                                html.Div(dcc.Dropdown(id="dropdown2",
                                                     options=region1option,value='New York'),style=
                                         {"display":"inline-block","border-style":"dashed",
                                        "text-align":"center","width":"26%","color":"black",
                                        "background-color":"yellow","border-color":"white"                                             
                                         }),
                                html.Div(dcc.Dropdown(id="dropdown3",
                                                     options=region2option,value='Bronx County'),style=
                                         {"display":"inline-block","border-style":"dashed",
                                        "text-align":"center","width":"26%","color":"black",
                                        "background-color":"red","border-color":"white"                                             
                                         }),        
                                html.Div(children=[html.H1(children="Date of Lockdown",style=
                                         {
                                            "display":"inline-block","font-size":"100%",
                                             "width":"10%","color":"black"
                                         }),
                                html.Div(dcc.DatePickerSingle(id='my-date-picker-single',
                                                             initial_visible_month=dt(2020,3,1),
                                                              date=str(dt(2020,3,1,23,59,59))
                                            ),style=
                                            {
                                                "display":"inline-block",
                                                "width":"10%","color":"black"
                                            })]),
                                html.Div(dcc.Graph(id="plot_area",figure={}))])

@app.callback(
                Output(component_id="plot_area",component_property="figure"),
                [Input(component_id="dropdown1",component_property="value"),
                Input(component_id="dropdown2",component_property="value"),
                Input(component_id="dropdown3",component_property="value"),
                 Input(component_id="my-date-picker-single",component_property="date"),
                ]
)


###collating and formatting the google mobility data
def updateplot(country,state,county,date):
    df=data_generator(country,state,county)
    date_lockdown = dt.strptime(re.split('T| ',date)[0],'%Y-%m-%d')
    categories_google_mobility = ['retail_and_recreation_percent_change_from_baseline',
                                 'grocery_and_pharmacy_percent_change_from_baseline',
                                 'parks_percent_change_from_baseline',
                                  'transit_stations_percent_change_from_baseline',
                                 'workplaces_percent_change_from_baseline',
                                  'residential_percent_change_from_baseline'
                                 ]
    minimum = min(list(df[categories_google_mobility].min()))
    maximum = max(list(df[categories_google_mobility].max()))
    
    plot_data=[]
    
    for group in categories_google_mobility:
        plot_data.append(go.Scatter(x=df['date'],y=df[group],name=group))
    
    if country==state and country==county:
        plot_layout=go.Layout(title=str(country))
    elif country==state:
        plot_layout=go.Layout(title=str(country)+'-'+str(state))
    else:
        plot_layout=go.Layout(title=str(country)+'-'+str(state)+'-'+str(county))
        
    fig = go.Figure(data=plot_data)
    fig.update_layout(plot_layout)
    fig.add_shape(
        #line vertical
        dict(
            type = "line",
            x0=date_lockdown,
            y0=maximum,
            x1=date_lockdown,
            y1=minimum,
            line=dict(
                    color="brown",
                    width=2
                )
))
    return fig

###generating data
def data_generator(country,state,county):
    submob_df = mob_df[(mob_df["country_region"]==country)&(mob_df["sub_region_1"]==state)&(mob_df["sub_region_2"]==county)]
    submob_df.loc[:,'date']=pd.to_datetime(submob_df.loc[:,'date'])
    submob_df = submob_df.sort_values('date',ascending=True)
    
    return submob_df
    
if __name__=='__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2021 00:54:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:54:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:54:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:54:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:55:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:55:07] "POST /_dash-update-component HTTP/1.1" 200 -
